In [1]:
from datasets.downloaders import read_addresses, CWRU_downloader
from datasets.digestors import CWRU
from signal_processing import *
from utils import *
import pandas as pd
import numpy as np

In [3]:
addresses = read_addresses()

downloader = CWRU_downloader('CWRU/', addresses['CWRU'])
downloader.download()

print(downloader.undownloaded)

while len(list(downloader.undownloaded.keys())) > 0:
    downloader.redownload()
    print(downloader.undownloaded)

Downloading: normal_1797_12K.mat
Error downloading normal_1797_12K.mat: ('Connection broken: IncompleteRead(2171457 bytes read, 1731887 more expected)', IncompleteRead(2171457 bytes read, 1731887 more expected))
Downloading: normal_1772_12K.mat
Error downloading normal_1772_12K.mat: ('Connection broken: IncompleteRead(5085253 bytes read, 2657467 more expected)', IncompleteRead(5085253 bytes read, 2657467 more expected))
Downloading: normal_1750_12K.mat
Error downloading normal_1750_12K.mat: ('Connection broken: IncompleteRead(3875982 bytes read, 11627946 more expected)', IncompleteRead(3875982 bytes read, 11627946 more expected))
Downloading: normal_1730_12K.mat
Downloaded: normal_1730_12K.mat
Downloading: DE_IR_007_1797_12K.mat
Downloaded: DE_IR_007_1797_12K.mat
Downloading: DE_IR_007_1772_12K.mat
Error downloading DE_IR_007_1772_12K.mat: ('Connection broken: IncompleteRead(2644380 bytes read, 283812 more expected)', IncompleteRead(2644380 bytes read, 283812 more expected))
Downloadin

In [4]:
mining_params = {
    '12K': {'win_len': 12000, 'hop_len': 3000},
    '48K': {'win_len': 48000, 'hop_len': 16000},
}

cwru = CWRU('CWRU/')
cwru.mine(mining_params, synchronous_only = True)

In [5]:
print(len(cwru.data['FE']['12K']), ' - ', len(cwru.data['FE']['48K']), '\n',
      len(cwru.data['DE']['12K']), ' - ', len(cwru.data['DE']['48K']))

101  -  52 
 101  -  52


In [6]:
df = pd.concat(cwru.data['DE']['12K']).reset_index(drop = True)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,11955,11956,11957,11958,11959,11960,11961,11962,11963,11964,11965,11966,11967,11968,11969,11970,11971,11972,11973,11974,11975,11976,11977,11978,11979,11980,11981,11982,11983,11984,11985,11986,11987,11988,11989,11990,11991,11992,11993,11994,11995,11996,11997,11998,11999,state,defected_bearing,severity,rot_speed,Fs
0,0.104365,0.625375,0.123045,-0.583142,0.009340,0.660299,-0.148222,-0.986793,-0.014619,1.066387,0.177866,-0.923444,-0.069441,1.184964,0.315530,-1.182934,-0.643243,0.996133,0.845881,-0.740298,-0.869840,0.577863,0.897454,-0.549843,-1.242223,0.259896,1.496434,0.151877,-1.254405,-0.105177,1.389633,0.419083,-1.253593,-0.719994,0.954306,0.750044,-0.784562,-0.802430,0.595325,0.857657,-0.469438,-1.070041,-0.024365,0.947809,0.334210,-0.638776,-0.277764,0.627000,0.333804,-0.561213,...,-0.709842,-1.112275,0.296850,1.222324,-0.065786,-1.553286,-0.380504,2.084449,1.003037,-2.173382,-1.819680,2.079170,2.437339,-1.546789,-2.682616,0.757354,2.819468,-0.072284,-2.341908,-0.592888,2.047089,1.326689,-1.308009,-1.772573,0.450757,2.205463,0.251774,-2.086073,-0.943342,1.869628,1.479784,-1.182122,-1.303136,0.765069,1.260497,-0.344769,-0.967707,-0.110456,0.552686,0.291165,-0.305784,-0.569335,-0.189237,0.470250,0.397154,OR@3,DE,007,1730,12K
1,0.450351,-0.270454,-0.548219,-0.094212,0.195328,0.004061,-0.019086,0.171775,0.143755,0.152283,0.266800,0.019492,-0.687507,-0.674106,0.467813,0.825170,-0.313094,-0.851566,0.492178,1.306385,-0.133197,-1.411967,-0.515732,0.961210,0.691568,-0.671263,-0.812582,0.531163,1.024966,-0.304972,-1.223137,-0.142943,1.127706,0.501112,-0.644055,-0.460098,0.286292,0.250150,-0.127105,0.007310,0.191673,0.038172,0.120608,0.488930,0.277764,-0.245683,-0.272079,0.023553,0.098679,-0.171775,...,-0.518574,0.532381,0.755323,-0.393499,-1.023747,0.168526,1.302730,0.110050,-1.554504,-0.738674,1.575215,1.232883,-1.420901,-1.730340,1.026184,1.941100,-0.627000,-2.011353,-0.104771,2.028003,0.953088,-1.294202,-1.373795,0.632685,1.686077,0.011370,-1.400191,-0.330962,1.437957,0.864155,-0.976235,-1.110650,0.521823,1.229634,-0.223754,-1.178873,-0.308221,0.919789,0.690349,-0.553498,-0.963240,-0.111268,0.960398,0.544564,-0.750450,OR@3,DE,007,1730,12K
2,0.408524,-0.661517,-0.638776,0.376443,0.802836,0.160811,-0.464971,-0.306596,0.292383,0.332180,-0.305378,-0.555934,0.001624,0.577051,0.328931,-0.343144,-0.402027,0.114923,0.288728,-0.077563,-0.316749,-0.025990,0.405682,0.360200,-0.049137,-0.262333,-0.166090,-0.084060,0.025584,0.337459,0.344362,-0.261114,-0.328119,0.652583,0.670857,-0.840196,-1.338872,0.549031,1.661305,-0.141725,-1.679173,-0.032487,2.047089,0.469438,-2.054398,-1.274710,1.527702,1.632879,-0.835323,-1.563032,...,0.418270,-0.046294,-0.344362,-0.207917,0.106801,0.187206,-0.047918,-0.227409,0.019898,0.423956,0.332992,-0.159999,-0.473092,-0.393905,-0.051979,0.301723,0.213602,-0.328931,-0.226597,0.737455,0.689537,-0.780095,-1.177655,0.411367,1.160599,-0.188831,-0.985169,0.343144,1.328313,-0.062131,-1.507398,-0.484057,1.119178,0.376443,-1.284456,-0.527508,1.526890,0.812176,-1.633285,-1.375013,1.535012,1.904552,-1.015220,-1.861100,0.983951,OR@3,DE,007,1730,12K
3,0.283043,0.846287,0.005279,-0.783343,-0.281825,0.552279,0.324870,-0.488930,-0.592076,0.059695,0.597355,0.415834,-0.257054,-0.578675,-0.010558,0.684258,0.266800,-0.763445,-0.744359,0.438169,0.926692,-0.043857,-0.865373,-0.274515,0.791465,0.549437,-0.555934,-0.614411,0.450757,0.794308,-0.216851,-0.808115,-0.156344,0.387814,-0.009340,-0.300099,0.140100,0.249744,-0.202638,-0.090151,0.387002,-0.004061,-0.796338,-0.420301,0.854409,0.808521,-0.541721,-0.504767,0.882023,0.865779,...,-0.211166,0.633091,0.378068,-0.489336,-0.494615,0.418677,0.778470,-0.029644,-0.732176,-0.315530,0.578675,0.493397,-0.458473,-0.623345,0.336241,0.761009,-0.122639,-0.677761,-0.062538,0.412585,0.056446,-0.189237,0.149440,0.268018,-0.054010,0.063756,0.341926,-0.159186,-0.

In [8]:
signals, metadata = df.iloc[:, : -5], df.iloc[:, -5 :]
signals

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,11950,11951,11952,11953,11954,11955,11956,11957,11958,11959,11960,11961,11962,11963,11964,11965,11966,11967,11968,11969,11970,11971,11972,11973,11974,11975,11976,11977,11978,11979,11980,11981,11982,11983,11984,11985,11986,11987,11988,11989,11990,11991,11992,11993,11994,11995,11996,11997,11998,11999
0,0.104365,0.625375,0.123045,-0.583142,0.009340,0.660299,-0.148222,-0.986793,-0.014619,1.066387,0.177866,-0.923444,-0.069441,1.184964,0.315530,-1.182934,-0.643243,0.996133,0.845881,-0.740298,-0.869840,0.577863,0.897454,-0.549843,-1.242223,0.259896,1.496434,0.151877,-1.254405,-0.105177,1.389633,0.419083,-1.253593,-0.719994,0.954306,0.750044,-0.784562,-0.802430,0.595325,0.857657,-0.469438,-1.070041,-0.024365,0.947809,0.334210,-0.638776,-0.277764,0.627000,0.333804,-0.561213,...,0.523041,-0.467001,-0.301317,0.786186,0.619690,-0.709842,-1.112275,0.296850,1.222324,-0.065786,-1.553286,-0.380504,2.084449,1.003037,-2.173382,-1.819680,2.079170,2.437339,-1.546789,-2.682616,0.757354,2.819468,-0.072284,-2.341908,-0.592888,2.047089,1.326689,-1.308009,-1.772573,0.450757,2.205463,0.251774,-2.086073,-0.943342,1.869628,1.479784,-1.182122,-1.303136,0.765069,1.260497,-0.344769,-0.967707,-0.110456,0.552686,0.291165,-0.305784,-0.569335,-0.189237,0.470250,0.397154
1,0.450351,-0.270454,-0.548219,-0.094212,0.195328,0.004061,-0.019086,0.171775,0.143755,0.152283,0.266800,0.019492,-0.687507,-0.674106,0.467813,0.825170,-0.313094,-0.851566,0.492178,1.306385,-0.133197,-1.411967,-0.515732,0.961210,0.691568,-0.671263,-0.812582,0.531163,1.024966,-0.304972,-1.223137,-0.142943,1.127706,0.501112,-0.644055,-0.460098,0.286292,0.250150,-0.127105,0.007310,0.191673,0.038172,0.120608,0.488930,0.277764,-0.245683,-0.272079,0.023553,0.098679,-0.171775,...,-0.263551,-0.204668,0.342332,0.132385,-0.625375,-0.518574,0.532381,0.755323,-0.393499,-1.023747,0.168526,1.302730,0.110050,-1.554504,-0.738674,1.575215,1.232883,-1.420901,-1.730340,1.026184,1.941100,-0.627000,-2.011353,-0.104771,2.028003,0.953088,-1.294202,-1.373795,0.632685,1.686077,0.011370,-1.400191,-0.330962,1.437957,0.864155,-0.976235,-1.110650,0.521823,1.229634,-0.223754,-1.178873,-0.308221,0.919789,0.690349,-0.553498,-0.963240,-0.111268,0.960398,0.544564,-0.750450
2,0.408524,-0.661517,-0.638776,0.376443,0.802836,0.160811,-0.464971,-0.306596,0.292383,0.332180,-0.305378,-0.555934,0.001624,0.577051,0.328931,-0.343144,-0.402027,0.114923,0.288728,-0.077563,-0.316749,-0.025990,0.405682,0.360200,-0.049137,-0.262333,-0.166090,-0.084060,0.025584,0.337459,0.344362,-0.261114,-0.328119,0.652583,0.670857,-0.840196,-1.338872,0.549031,1.661305,-0.141725,-1.679173,-0.032487,2.047089,0.469438,-2.054398,-1.274710,1.527702,1.632879,-0.835323,-1.563032,...,0.268424,-0.047918,-0.358982,-0.101928,0.387002,0.418270,-0.046294,-0.344362,-0.207917,0.106801,0.187206,-0.047918,-0.227409,0.019898,0.423956,0.332992,-0.159999,-0.473092,-0.393905,-0.051979,0.301723,0.213602,-0.328931,-0.226597,0.737455,0.689537,-0.780095,-1.177655,0.411367,1.160599,-0.188831,-0.985169,0.343144,1.328313,-0.062131,-1.507398,-0.484057,1.119178,0.376443,-1.284456,-0.527508,1.526890,0.812176,-1.633285,-1.375013,1.535012,1.904552,-1.015220,-1.861100,0.983951
3,0.283043,0.846287,0.005279,-0.783343,-0.281825,0.552279,0.324870,-0.488930,-0.592076,0.059695,0.597355,0.415834,-0.257054,-0.578675,-0.010558,0.684258,0.266800,-0.763445,-0.744359,0.438169,0.926692,-0.043857,-0.865373,-0.274515,0.791465,0.549437,-0.555934,-0.614411,0.450757,0.794308,-0.216851,-0.808115,-0.156344,0.387814,-0.009340,-0.300099,0.140100,0.249744,-0.202638,-0.090151,0.387002,-0.004061,-0.796338,-0.420301,0.854409,0.808521,-0.541721,-0.504767,0.882023,0.865779,...,-0.198171,0.431671,0.609132,-0.157562,-0.771973,-0.211166,0.633091,0.378068,-0.489336,-0.494615,0.418677,0.778470,-0.029644,-0.732176,-0.315530,0.578675,0.493397,-0.458473,-0.623345,0.336241,0.761009,-0.1

In [9]:
metadata

,state,defected_bearing,severity,rot_speed,Fs
0,OR@3,DE,007,1730,12K
1,OR@3,DE,007,1730,12K
2,OR@3,DE,007,1730,12K
3,OR@3,DE,007,1730,12K
4,OR@3,DE,007,1730,12K
...,...,...,...,...,...
4136,OR@3,FE,007,1730,12K
4137,OR@3,FE,007,1730,12K
4138,OR@3,FE,007,1730,12K
4139,OR@3,FE,007,1730,12K


In [10]:
metadata['comb'] = metadata['state'] + '_' + metadata['defected_bearing'] + '_' + metadata['severity'] + '_' + metadata['rot_speed']
metadata['comb'].value_counts()

comb
normal_-_-_1750      158
normal_-_-_1730      158
normal_-_-_1772      158
normal_-_-_1797       78
OR@3_FE_014_1750      37
                    ... 
OR@6_FE_007_1750      37
IR_DE_014_1797        37
OR@12_DE_021_1730     37
Ball_DE_014_1750      37
OR@3_FE_007_1730      37
Name: count, Length: 101, dtype: int64

# Envelope Extraction

In [11]:
signals_env = Env(signals)
signals_env.shape

(4141, 12000)

# FFT

In [13]:
window = scipy.signal.windows.hann(signals_env.shape[1])
freq_filter = scipy.signal.butter(25, [5, 5500], 'bandpass', fs = 12000, output='sos')

In [14]:
signals_env_fft = FFT(signals_env, freq_filter = freq_filter, window = window)
signals_env_fft.shape

(4141, 6000)

# ZoomedFFT

In [15]:
signals_env_ZoomedFFT = ZoomedFFT(signals_env, 0, 1000, 2000, 12000, freq_filter = freq_filter, window = window)
signals_env_ZoomedFFT.shape

(4141, 2000)

# STFT

In [16]:
STFT_window = scipy.signal.windows.hann(1024)
STFT_freq_filter = scipy.signal.butter(25, [5, 5500], 'bandpass', fs = 12000, output='sos')
signals_env_STFT = STFT(signals_env, 1024, 200, STFT_freq_filter, STFT_window)
signals_env_STFT.shape

(4141, 55, 512)

# Statistical Features

In [17]:
feature(signals, {'mean': np.mean, 'var': np.var, 'rms': rms})

,mean,var,rms
0,0.007676,0.623656,0.789756
1,0.007450,0.621862,0.788617
2,0.007274,0.617593,0.785904
3,0.007451,0.617107,0.785597
4,0.007175,0.614402,0.783871
...,...,...,...
4136,0.002539,0.009192,0.095910
4137,0.002579,0.009194,0.095920
4138,0.002579,0.009407,0.097024
4139,0.002570,0.009416,0.097071


# Visualization

In [18]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1, subplot_titles=("Original Time Signal", "Envelope", "FFT", "Zoomed FFT"))

fig.append_trace(go.Scatter(
    y=signals.iloc[0,:],
), row=1, col=1)

fig.append_trace(go.Scatter(
    y=signals_env.iloc[0, :],
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=fft_freq_axis(10000, 12000),
    y=signals_env_fft.iloc[0, :]
), row=3, col=1)

fig.append_trace(go.Scatter(
    x=ZoomedFFT_freq_axis(0, 1000, 2000),
    y=signals_env_ZoomedFFT.iloc[0, :]
), row=4, col=1)

fig.update_layout(height=800, width=1500, title_text="Stacked Subplots")
fig.show()

In [19]:
import plotly.express as px

t = np.linspace(0, 1, 55)
f = fft_freq_axis(1024, 12000)

fig = px.imshow(signals_env_STFT[0, :, :], x = f, y = t, aspect="auto")
fig.show()